## Fig 12: Impedance match calculation simulations

Input files to re-create simulations in Fig. 12.

pykodt.py is modified to force smaller time steps to resolve the plume decompression path.

pyKO documentation at https://github.com/StewartGroup/ko-hydrocode

Sarah T. Stewart<br>
February 7, 2025<br>

In [14]:
%run ../import-modules-grid # adjust path and use pykodt.py

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

!jupyter --version

import colormaps as local_cmaps

Platform:  Darwin Kernel Version 24.2.0: Fri Dec  6 19:02:41 PST 2024; root:xnu-11215.61.5~2/RELEASE_ARM64_T6030
python version:  3.11.10
matplotlib version:  3.9.2
hvplot version:  0.11.0
numpy version:  2.1.2
pandas version:  2.2.3
pickle version:  4.0
yaml version:  6.0.2
pint version:  0.24.3
pyko version:  v0.8.3-dev-2024-05-12-dt
print eos_table version:  v1.1.5b

Number of CPUs in the system: 12


Selected Jupyter core packages...
IPython          : 8.28.0
ipykernel        : 6.29.5
ipywidgets       : 8.1.5
jupyter_client   : 8.6.3
jupyter_core     : 5.7.2
jupyter_server   : 2.14.2
jupyterlab       : 4.2.5
nbclient         : 0.10.0
nbconvert        : 7.16.4
nbformat         : 5.10.4
notebook         : not installed
qtconsole        : not installed
traitlets        : 5.14.3


## Example of checking EOS table initial state and verifying input file OK

EOS Tables are initialized based on the user input density and internal energy (you could change that if desired). In v0.4, the input P and T are over-ridden by the EOS Table state variables for the given density and energy.

In [15]:
###%%capture cap
rplumeinitarr = np.asarray([25.e3]) # m
rnebinitarr = np.asarray([100.e6]) #m
tsavearr = np.asarray([10.e-6]) #s
tsavearr = np.asarray([5.e-6,5.e-6,5.e-6]) #s
pinitarr = np.asarray([40.e9,20.e9,10.e9])
#tendarr = np.asarray([500,1000,1500])*1.e6
#tendarr = np.asarray([500,1000,1500])*1.e6
tendarr = np.asarray([500,1000,1500])
velinitarr = np.asarray([0.]) # m/s

pinitarr = np.asarray([40.e9,20.e9,10.e9])
tendarr = np.asarray([10,10,10])*1.e6 # s


In [16]:

if 0: # change to 1 to re-run the calculations
    fin='vp-h2o-exp-dg1.yml' # set path
    fout='vp-h2o-exp2-dg1-' # set path
    #tend = 86400.e6*15 # initial stop time
    #tend = 1000e6 # initial stop time
    ftype='YAML'
    verbose=True
    userdtstart=100. # microsec
    usertstepscale=1
    binoutput=True
    debug=False
    initarr=False
    run = RunClass(fin=fin,fout=fout,ftype=ftype)    # initialize run parameters class
    #
    # read in the run template
    readinput_yaml(run,verbose=verbose)
    print(run.ieos[0].path)
    hugarr = np.loadtxt(run.ieos[0].path+'NEW-SESAME-HUG-E.TXT',skiprows=3,delimiter=',')
    print('Hugoniot file size = ',hugarr.shape)
    # vary starting parameters
    for ipp in range(len(pinitarr)):
        tend = tendarr[ipp]
        # figure out the initial state on the principal Hugoniot
        rhostart = np.interp(pinitarr[ipp]/1.e9,hugarr[:,2],hugarr[:,1])
        iestart =  np.interp(pinitarr[ipp]/1.e9,hugarr[:,2],hugarr[:,3])
        tempstart =  np.interp(pinitarr[ipp]/1.e9,hugarr[:,2],hugarr[:,0])
        entstart =  np.interp(pinitarr[ipp]/1.e9,hugarr[:,2],hugarr[:,4])
        upplanarstart =  np.interp(pinitarr[ipp]/1.e9,hugarr[:,2],hugarr[:,5])
        print('initial state iii, P, rho, ie, T [GPa, g/cm3, MJ/kg] = ',ipp,pinitarr[ipp]/1.e9,rhostart,iestart,tempstart)
        print('initial state iii, S [J/K/kg] = ',entstart*1.e3)
        print('equivalent planar up, planar vi (km/s) = ',upplanarstart,upplanarstart*2)
        print('total mass [kg] = ',rhostart*1000*4./3*3.14159*np.power(rplumeinitarr,3))
        print('total IE [MJ] = ',rhostart*1000*4./3*3.14159*np.power(rplumeinitarr,3)*iestart)
        ureg.define('eu = 1.0E12 ergs') 
        utmp = Q_(iestart,'MJ/kg') # creates a variable utmp that has pint units information
        print('internal energy in code units eu/g: ',utmp.to('eu/g')) # prints the value and units
        print('internal energy in mks units: ',utmp.to('J/kg')) # prints the value and units
        ptmp = Q_(pinitarr[ipp]/1.e9,'GPa') # creates a variable ptmp that has pint units information
        print('pressure in code units Mbar: ',ptmp.to('Mbar')) # prints the value and units
        print('pressure in mks (Pa) units: ',ptmp.to('Pa')) # prints the value and units
        print('pressure in mks (GPa) units: ',ptmp.to('GPa')) # prints the value and units
        #initarr = [upstart,rhostart,pstart,tstart,ievstart,csstart] # code units
        rtmp = Q_(rhostart,'g/cm^3')
        for irp in range(len(rplumeinitarr)):
            for ivel in range(len(velinitarr)):            
                run = RunClass(fin=fin,fout=fout,ftype=ftype)    # initialize run parameters class
                readinput_yaml(run,verbose=verbose)
                fileid = 'P'+str(np.round(pinitarr[ipp]/1.e9))+'-R'+str(np.round(rplumeinitarr[irp]/1.e3))+'-V'+str(np.round(velinitarr[ivel]/1.e2)/10)
                run.outputfilename = fout+fileid+'.dat'
                print('outputfile',run.outputfilename)
                initarr = [velinitarr[ivel]/10.e3,rtmp.magnitude,ptmp.to('Mbar').magnitude,utmp.to('eu/g').magnitude*rtmp.magnitude,tempstart]
                print(initarr)
                run.ixstart[0]=rplumeinitarr[irp]*1.e2/5 # cm
                run.ixstart[1]=rplumeinitarr[irp]*1.e2 # cm
                run.ilength[0]=rplumeinitarr[irp]*1.e2-rplumeinitarr[irp]*1.e2/5# cm
                run.ilength[1]=rnebinitarr[irp]*1.e2 # cm
                print('length vars = ',run.ixstart,run.ilength,run.inodes)
                run.time_skip = tsavearr[irp]*1.e6 # microsec
                print('Save output every microsec =',run.time_skip)
                
                ##############################################################
                if 1:
                    print('\n pyKO STARTING RUN \n')
                    run.tstop=tend
                    # initialize problem domain data structure
                    # pass in the problem run parameters
                    data = DomainClass() # create main data structure
                    # set the initial velocity of the vapor plume
                    # using code units for now; velocity units are 10 km/s
                    if initarr != False:
                        #initarr = [upstart,rhostart,pstart,ievstart,tempstart] # code units
                        run.iupstart[0]=initarr[0]
                        run.irhostart[0]=initarr[1]
                        run.ipstart[0]=initarr[2]
                        run.iiev0start[0]=initarr[3]            
                        run.itempstart[0]=initarr[4]
                        # not filling in sound speed
                        run.ieos[0].p0     = initarr[2]
                        run.ieos[0].up0    = initarr[0]
                        run.ieos[0].rho0   = initarr[1]
                        run.ieos[0].iev0   = initarr[4]
                        run.ieos[0].EOS.R0REF = initarr[1]
                        #run.outputfilename = run.outputfilename+'-'+str(initarr[0])+'-'+str(round(initarr[2]*100))
                    print('initial up = ',run.iupstart)
                    print('initial p = ',run.ipstart)
                    print('initial rho = ',run.irhostart)
                    print('initial ie = ',run.iiev0start)
                    print('outputfile',run.outputfilename)
                    data.makegrid(run,verbose=verbose) # create initial grid; also calculates a guess for the initial time step
                    if userdtstart > 0.:
                        # for testing comparisons to fKO with fixed initial time step
                        if verbose: print('Using user supplied initial time step (microsec): ',userdtstart)
                        run.dtstart = userdtstart
                        data.deltat   = run.dtstart      # current time step; while in time loop
                        data.deltat_next = run.dtstart   # next time step; calculated at end of time loop
                    if usertstepscale > 0.:
                        if verbose: print('Using user supplied reduction factor for time steps: ',usertstepscale)
                        run.tstepscale = usertstepscale
                    run.debugflag = debug
                    data.writefirstoutput(run,checkexistsflag=False,verbose=verbose,overwriteflag=True) # write initial grid to outputfile - right now just the header
                    
                    # testing: can call one time step only or a fixed number of time steps to check the simulation parameters
                    # advance one time step
                    #data.advancetime(run)
                    #for steps in range(20):
                    #    data.advancetime(run,verbose=verbose)
                        
                    # main time loop
                    print('pyKO is running.....')
                    t0 = pytime.time()
                    # run once
                    if 0:
                        data.advancetime(run,verbose=verbose)
                    if 1:
                        runflag=True
                        flag1 = True
                        flag2 = True
                        flag3 = True
                        while data.time[3] < run.tstop:
                        #while runflag:
                            data.advancetime(run,verbose=verbose)
                            mat1negup = np.where((data.up[2,10::] < -10.e-4))[0] # < 1 m/s
                            #imat1max = np.max(np.where(data.matid ==0)[0])
                            #if runflag and (data.up[2,imat1max] < 0.):
                            #rint(mat1negup)
                            if 0:
                                if (data.time[3]>5.e6) & (flag1):
                                    run.time_skip = tsavearr[irp]*1.e6*1000. 
                                    flag1=False
                                    print(data.time[3],run.time_skip,run.tstepscale)
                                    run.tstepscale = 3
                                if (data.time[3]>50.e6) & (flag2):
                                    run.time_skip = tsavearr[irp]*1.e6*1000000.
                                    flag2=False
                                    run.tstepscale = 1
                                    print(data.time[3],run.time_skip,run.tstepscale)
                                if (data.time[3]>100.e6) & (flag3):
                                    run.time_skip = tsavearr[irp]*1.e6*100000000.
                                    flag3=False
                                    print(data.time[3],run.time_skip,run.tstepscale)
                                    
                            if runflag and (len(mat1negup)>0):
                                runflag=False
                                print(mat1negup)
                                print(data.up[2,0:201])
                                # microsec to sec; cm to Mm
                                print('STALL TIME (s), position (Mm) = ',data.time[3]/1.e6,data.pos[2,10::][mat1negup[0]]/1.e2/1.e6,data.stepn)
                                data.onebinaryoutputpint(run,filetag='-stall')
                                run.tstop = data.time[3]*3.
                    #
                    if data.stepn > max(run.outputsteps): # write last time step if needed
                        data.binaryoutputpint(run) # pickle output 
                    t1 = pytime.time()
                    
                    # diagnostic energy checks and run time information
                    print('Zeroth and final mvtotal: ',run.outputmvtot[0],run.outputmvtot[-1])
                    print('Zeroth and final KE+IE: ', \
                          run.outputietot[0]+run.outputketot[0],run.outputietot[-1]+run.outputketot[-1])
                    print('KE+IE ratio (final/zeroth)):', \
                          (run.outputietot[-1]+run.outputketot[-1])/(run.outputietot[0]+run.outputketot[0]))
                    print('First output step and final output step KE+IE: ', \
                          run.outputietot[1]+run.outputketot[1],run.outputietot[-1]+run.outputketot[-1])
                    print('KE+IE ratio (final/first)):', \
                          (run.outputietot[-1]+run.outputketot[-1])/(run.outputietot[1]+run.outputketot[1]))
                    print('\n pyKO FINISHED RUN, simulation time = ',data.time[1+2],' us \n')
                    print(' pyKO Wall Clock Main Loop Run Time=', t1-t0, ' s')
                    print(run.outputfilename)
                    #stop
                  


Fracture is turned off for mat1
Fracture is turned off for mat2
found pvoid

pyKO v0.8.3-dev-2024-05-12-dt run parameters
   All outputs are in code units 
   Input file: ./vp-h2o-expansion-early/vp-h2o-exp-dg1.yml 
   Output file: ./vp-h2o-expansion/grid2- 
   Number of materials: 2 
   Number of nodes in each material: [ 50 500] 
   Length of each material: [2.5e+06 1.0e+10] 
   Initial left edge of each material: [      0. 2500000.] 
   Boundary conditions: ['FIXED', 'FIXED']
   Material EOS:     ['SES', 'DIG'] 
   Geometry:         SPH 
   Gravity:          0.0 
   Void pressure:    1.0000000000000002e-12 
   Time step factor: 6 
   Stop time:        1000000.0

mat1: Hydrodynamic material

mat1 Fracture parameters [code units]: 
   Fracture is turned on:                       False 
   Fracture pressure:                           1e+99 
   Fracture maximum distension (rhomin/rhoref): 0.8

Class SESAME [code units]: ANEOS H2O 
   eos_table module version v1.1.5b 
   table path: ../a

In [17]:
def pyko_to_normal_panda(pkodata):
    jmaxmat1=np.max(np.where(pkodata.mat.magnitude ==1)[0])
    pkodata.temp[jmaxmat1-5:jmaxmat1+1] = pkodata.temp[jmaxmat1-6]
    pkodata.temp[jmaxmat1+1:jmaxmat1+7] = pkodata.temp[jmaxmat1+7]
    pkodata.rho[jmaxmat1-5:jmaxmat1+1] = pkodata.rho[jmaxmat1-6]
    pkodata.rho[jmaxmat1+1:jmaxmat1+7] = pkodata.rho[jmaxmat1+7]
    df = pd.DataFrame({
            "j"    : pkodata.j.magnitude,
            "stepn" : pkodata.stepn.magnitude,
            "time" : pkodata.time.magnitude,
            "mat" : pkodata.mat.magnitude,
            "pos" : pkodata.pos.magnitude,
            "dr" : pkodata.dr.magnitude,
            "rho0" : pkodata.rho0.magnitude,
            "rho" : pkodata.rho.magnitude,
            "up" : pkodata.up.magnitude,
            "ie" : pkodata.ie.magnitude,
            "pres" : pkodata.pres.magnitude,
            "mass" : pkodata.mass.magnitude*4*np.pi, # convert to true total mass
            "temp" : pkodata.temp.magnitude,
            "cs" : pkodata.alocal.magnitude,
            "phase" : pkodata.phase.magnitude,
            "etot" : pkodata.etot.magnitude,
            "dtminj" : pkodata.dtminj.magnitude,
            })
    return df
def pyko_to_pos_panda(pkodata,jmaxmat1):
    posdata = (np.arange(15000)+1)*2*1.E3 #m
    df = pd.DataFrame({
            "time" : np.interp(posdata,pkodata.pos.magnitude,pkodata.time.magnitude)/3600,
            "mat" : np.interp(posdata,pkodata.pos.magnitude,pkodata.mat.magnitude),
            "rho" : np.log10(np.interp(posdata,np.concatenate((pkodata.pos.magnitude[2:jmaxmat1-2], pkodata.pos.magnitude[jmaxmat1+3::])),np.concatenate((pkodata.rho.magnitude[2:jmaxmat1-2], pkodata.rho.magnitude[jmaxmat1+3::])))/1.e3), # log10 g/cm3
            "pres" : np.log10(np.interp(posdata,np.concatenate((pkodata.pos.magnitude[2:jmaxmat1-2], pkodata.pos.magnitude[jmaxmat1+3::])),np.concatenate((pkodata.pres.magnitude[2:jmaxmat1-2], pkodata.pres.magnitude[jmaxmat1+3::])))/1.e5), # log10 bar
            "up" : np.interp(posdata,np.concatenate((pkodata.pos.magnitude[2:jmaxmat1-2], pkodata.pos.magnitude[jmaxmat1+3::])),np.concatenate((pkodata.up.magnitude[2:jmaxmat1-2], pkodata.up.magnitude[jmaxmat1+3::]))), # m/s
            "temp" : np.interp(posdata,np.concatenate((pkodata.pos.magnitude[2:jmaxmat1-2], pkodata.pos.magnitude[jmaxmat1+3::])),np.concatenate((pkodata.temp.magnitude[2:jmaxmat1-2], pkodata.temp.magnitude[jmaxmat1+3::]))),
            "pos" : posdata/1.e3 # km
            })
    return df
def pyko_to_pos_panda_clean(pkodata):
    posdata = (np.arange(25000)+1)*4*1.E3 #m
    df = pd.DataFrame({
            "time" : np.interp(posdata,pkodata.pos.magnitude,pkodata.time.magnitude)/3600,
            "mat" : np.interp(posdata,pkodata.pos.magnitude,pkodata.mat.magnitude),
            "rho" : np.log10(np.interp(posdata,pkodata.pos.magnitude,pkodata.rho.magnitude)*1.e-3), # log_10 of g/cm3
            "pres" : np.log10(np.interp(posdata,pkodata.pos.magnitude,pkodata.pres.magnitude)/1.e5), # log_10 of bar
            "up" : np.interp(posdata,pkodata.pos.magnitude,pkodata.up.magnitude), # m/s
            "temp" : np.interp(posdata,pkodata.pos.magnitude,pkodata.temp.magnitude), # K 
            "pos" : posdata/1.e3 # km
            })
    return df
def pyko_to_pos_panda_track(pkodata):
    npos = 1000
    maxrad = np.max(pkodata.pos.magnitude) # m
    posdata = (np.arange(npos)+1)/npos*maxrad  #m
    jmaxmat1=np.max(np.where(pkodata.mat.magnitude ==1)[0])
    pkodata.temp[jmaxmat1-5:jmaxmat1+1] = pkodata.temp[jmaxmat1-6]
    pkodata.temp[jmaxmat1+1:jmaxmat1+6] = pkodata.temp[jmaxmat1+6]
    pkodata.rho[jmaxmat1-5:jmaxmat1+1] = pkodata.rho[jmaxmat1-6]
    pkodata.rho[jmaxmat1+1:jmaxmat1+6] = pkodata.rho[jmaxmat1+6]
    df = pd.DataFrame({
            "time" : np.zeros(npos)+pkodata.time[0].magnitude/3600.,#np.interp(posdata,pkodata.pos.magnitude,pkodata.time.magnitude)/3600, # hr
            "mat" : np.round(np.interp(posdata,pkodata.pos.magnitude,pkodata.mat.magnitude)),
            "rho" : np.log10(np.interp(posdata,pkodata.pos.magnitude,pkodata.rho.magnitude)), # log_10 of kg/m3
            "pres" : np.log10(np.interp(posdata,pkodata.pos.magnitude,pkodata.pres.magnitude)), # log_10 of Pa
            "up" : np.interp(posdata,pkodata.pos.magnitude,pkodata.up.magnitude), # m/s
            "temp" : np.interp(posdata,pkodata.pos.magnitude,pkodata.temp.magnitude), # K 
            "pos" : posdata/1.e6# Mm
            })
    return df

In [18]:
#rplumeinitarr = np.asarray([25.e3]) # m

maxtime = 86400

all_label = []
all_tmax = []
all_pmax = []
all_cvol = []
all_rstall = []
all_plumeIM = []
all_nebIM = []

if 0: #set to 1 to re-run
    fin='vp-h2o-exp-dg1.yml' # set path
    fout='vp-h2o-exp2-dg1-' # set path

    ftype='YAML'
    verbose=True
    userdtstart=0.
    usertstepscale=0.
    binoutput=True
    debug=False
    initarr=False
    #run = RunClass(fin=fin,fout=fout,ftype=ftype)    # initialize run parameters class
    #
    # read in the run template
    #readinput_yaml(run,verbose=0)
    #print(run.ieos[0].path)
    #hugarr = np.loadtxt(run.ieos[0].path+'NEW-SESAME-HUG-E.TXT',skiprows=3,delimiter=',')
    #print('Hugoniot file size = ',hugarr.shape)
    # vary starting parameters
    for ipp in range(len(pinitarr)):
#    for ipp in [0]:
        for irp in range(len(rplumeinitarr)):
            for ivel in range(len(velinitarr)):            
                fileid = 'P'+str(np.round(pinitarr[ipp]/1.e9))+'-R'+str(np.round(rplumeinitarr[irp]/1.e3))+'-V'+str(np.round(velinitarr[ivel]/1.e2)/10)
                #fileid='cth5-ring'
                outputfilename = fout+fileid+'.dat'
                label = 'dg1-'+fileid[0:-7]
                print("#outputfilename='"+outputfilename+"'")
                pykofileout = outputfilename
                pko = [] # this variable will hold a plain (no units) pandas datafram for plotting
                pkopos = []
                pkoposclean = []
                pkopostrack = []
                pkodata = OutputClass() # pandas + pint dataframe to read the pickled output data
                #
                # function to convert the stored pandas structure with pint units to a normal panda file
                # hvplot tools do not work with a panda+pint file
                # this also lets the user select a subset of variables to read into this notebook
                
                #
                # loop through all the pickle dumps to read in the simulation data
                # concat onto a pandas dataframe that stores the variables vs. time
                with open(pykofileout,"rb") as f:
                    pkodata = pickle.load(f) # keeps units
                    if 0:
                        # print units
                        print('pyKO output file units are the same as the input file units:')
                        print('   Time        ',pkodata.time.units)
                        print('   Position    ',pkodata.pos.units)
                        print('   Density     ',pkodata.rho.units)
                        print('   Part. vel.  ',pkodata.up.units)
                        print('   Int. energy ',pkodata.ie.units)
                        print('   Mass        ',pkodata.mass.units)
                        print('   Temperature ',pkodata.temp.units)
                        print('   Sound speed ',pkodata.alocal.units)
                        print('   Pressure    ',pkodata.pres.units)
                        print('   Stress      ',pkodata.sigmar.units)
                    #jmaxmat1=max(np.where(pkodata.mat.magnitude==1)[0])
                    pko = pyko_to_normal_panda(pkodata)
                    jmaxmat1=max(pko['j'][pko['time']==0][pko['mat']==1])
                    pkopos = pyko_to_pos_panda(pkodata,jmaxmat1)
                    pkoposclean = pyko_to_pos_panda_clean(pkodata)
                    pkopostrack = pyko_to_pos_panda_track(pkodata)
                    count=0
                    modval=100
                    while True:
                        try:
                            pkodata = pickle.load(f) # keeps units but only one snapshot at a time
                            if (count % modval)==0:
                                pko = pd.concat([pko,pyko_to_normal_panda(pkodata)],ignore_index=True) # strips units for plotting
                                #pkopos = pd.concat([pkopos,pyko_to_pos_panda(pkodata,jmaxmat1)],ignore_index=True) # strips units for plotting
                                #pkoposclean = pd.concat([pkoposclean,pyko_to_pos_panda_clean(pkodata)],ignore_index=True) # strips units for plotting
                                #pkopostrack = pd.concat([pkopostrack,pyko_to_pos_panda_track(pkodata)],ignore_index=True) # strips units for plotting
                            count=count+1
                            #print(count)
                            if max(pko['time']>maxtime):
                                break
                        except:
                            break
                timeall = np.unique(pko['time'])/3600.
    
                pko = pko.assign(material = pko.mat)
                imat1 = np.where((pko['mat'] == 1))[0]
                #pko['material'][imat1]=1
                imat2 = np.where((pko['mat'] == 2))[0]
                #pko['material'][imat2]=2
                jmaxmat1=max(pko['j'][pko['time']==0][pko['mat']==1])
                
                # convert to same units as fKO for plot comparisons
                # from binary in mks
                #pko['ie']     *= 1.E-11*pko['rho0']    # J/kg * kg/m3 -> 100 GJ/m3 = eu/cm3
                #pko.rename(columns={"ie": "iev0"},inplace=True)
                pko['etot']   *= 1.E-8    # J/kg 10e7 erg/1000 g -> erg/g *1.e-12 -> eu/g
                #print('iev0 and etot converted to eu/g')
                pko['time']   /= 3600    # s-> hr
                pko['dtminj'] *= 1.0E6    # s->microseconds
                pko['pos']    *= 1.0E-6    # m->Mm
                #pko['pres']   *= 1.E-5    # Pa -> bar
                pko['rho']    *= 1.E-3    # kg/m3 -> g/cm3
                pko['rho0']   *= 1.E-3    # kg/m3 -> g/cm3
    
                timeall = np.unique(pko['time'])
                

                jmaxmat1=max(pko['j'][pko['mat']==1])
                plupko = pko[['pres', 'up','rho','temp','j','time','pos']][pko['j']==jmaxmat1].copy()
                nebpko = pko[['pres', 'up','rho','temp','j','time','pos']][pko['j']==jmaxmat1+2].copy()
                plupko.to_csv(outputfilename[0:-4]+'-IM.csv')

                all_plumeIM.append(plupko)
                all_nebIM.append(nebpko)
                
##                %run plot-save-three.ipynb
##                %run plot-save.ipynb
##                stop

#outputfilename='./vp-h2o-expansion-early/vp-h2o-exp2-dg1-P40.0-R25.0-V0.0.dat'
#outputfilename='./vp-h2o-expansion-early/vp-h2o-exp2-dg1-P20.0-R25.0-V0.0.dat'
#outputfilename='./vp-h2o-expansion-early/vp-h2o-exp2-dg1-P10.0-R25.0-V0.0.dat'


End of notebook